In [ ]:
import scipy.io as sio
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn
from torchvision.datasets import SVHN

In [38]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [39]:
class SVHNDataset(Dataset):
    def __init__(self, mat_file, transform=None):
        self.data = sio.loadmat(mat_file)
        self.images = self.data['X'].transpose(3, 0, 1, 2)  # Transpose to (N, H, W, C)
        self.labels = self.data['y'].squeeze()  # Remove extra dimension
        self.labels[self.labels == 10] = 0  # Change label 10 to 0 for digit 0
        self.transform = transform

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        image = Image.fromarray(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [40]:
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16*5*5, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10) 
        )
    
    def forward(self, x):
        return self.net(x)

In [45]:
epochs = 25
model = LeNet5()
criterion = nn.CrossEntropyLoss()
lr=0.001
optimizer = torch.optim.Adam(model.parameters(), lr)

In [46]:
train_dataset = SVHNDataset('/Users/shamsbenmefteh/Documents/Fine_tuning_exp/data/SVHN/train_32x32.mat', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)

test_dataset = SVHNDataset('/Users/shamsbenmefteh/Documents/Fine_tuning_exp/data/SVHN/test_32x32.mat', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=6, shuffle=False)

In [ ]:
def train(model, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):  # 5 epochs as example
        model.train()
        running_loss, val_acc = 0.0, 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, val_acc={val_acc/len(train_loader):.4f}")

Epoch 1, Loss: 0.6773
Epoch 2, Loss: 0.4426
Epoch 3, Loss: 0.3973
Epoch 4, Loss: 0.3699
Epoch 5, Loss: 0.3497
Epoch 6, Loss: 0.3374
Epoch 7, Loss: 0.3277
Epoch 8, Loss: 0.3236
Epoch 9, Loss: 0.3149
Epoch 10, Loss: 0.3055
Epoch 11, Loss: 0.3053
Epoch 12, Loss: 0.2999
Epoch 13, Loss: 0.2954
Epoch 14, Loss: 0.2945
Epoch 15, Loss: 0.2901
Epoch 16, Loss: 0.2878
Epoch 17, Loss: 0.2888
Epoch 18, Loss: 0.2875
Epoch 19, Loss: 0.2875
Epoch 20, Loss: 0.2859
Epoch 21, Loss: 0.2797
Epoch 22, Loss: 0.2852
Epoch 23, Loss: 0.2824
Epoch 24, Loss: 0.2784
Epoch 25, Loss: 0.2760


In [ ]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss, test_acc = 0.0, 0
    with torch.no_grad():
        for x, y in test_loader:
            preds = model(x)
            test_loss += criterion(preds, y).item()
            test_acc  += (preds.argmax(1)==y).sum().item()
    print(f"Test   : loss={test_loss/len(test_loader):.4f}, acc={test_acc/len(test_dataset):.4f}")

Test   : loss=0.5253, acc=0.8509


In [ ]:
train(model, train_loader, criterion, optimizer, epochs)
test(model, test_loader, criterion)